<a href="https://colab.research.google.com/github/idutsu/kirikuchikun-ai/blob/main/src/search-word-wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 必要なライブラリをインストール
!apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3 gensim tqdm
!pip install unidic
!python -m unidic download

# 必要なモジュールをインポート
import os
import json
import MeCab
from tqdm import tqdm
from gensim.models import KeyedVectors

# Word2Vecモデルをロード (学習済みモデルのパス)
word2vec_model_path = "/content/drive/MyDrive/kirikuchikun-ai/jawiki.all_vectors.200d.txt"  # 事前にアップロードしたパスに変更してください
print("Word2Vecモデルをロード中...")
word_vectors = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=False)
print("Word2Vecモデルのロードが完了しました。")

# MeCab初期化
mecab = MeCab.Tagger()

# ベースディレクトリと検索条件
base_directory = "/content/drive/MyDrive/kirikuchikun-ai/wikipedia_json_output"  # JSONファイルが格納されているディレクトリ


# 形態素解析して、指定した品詞を抽出する関数
def extract_words(text):
    """
    テキストを形態素解析して名詞、形容詞、動詞、副詞を抽出する。
    """
    allowed_pos = ["名詞", "形容詞", "動詞", "副詞"]
    parsed = mecab.parse(text).splitlines()
    words = []
    for line in parsed:
        if "\t" in line:
            surface, features = line.split("\t")
            pos = features.split(",")[0]  # 品詞
            if pos in allowed_pos:
                words.append(surface)
    return words

# Word2Vecで類似度を計算する関数
def calculate_similarity(word, target_word, word_vectors):
    try:
        return word_vectors.similarity(word, target_word)
    except KeyError:
        return None  # モデルに単語が存在しない場合

# JSONファイルを探索して処理する関数
def search_title_and_calculate_similarity(base_dir, exact_title, target_word):
    for root, _, files in os.walk(base_dir):
        json_files = [f for f in files if f.endswith(".json")]
        for json_file in tqdm(json_files, desc=f"処理中: {root}"):
            file_path = os.path.join(root, json_file)
            try:
                with open(file_path, "r", encoding="utf-8") as infile:
                    for line in infile:
                        if not line.strip():
                            continue
                        try:
                            data = json.loads(line)
                            title = data.get("title", "")
                            text = data.get("text", "")

                            if title == exact_title:
                                print(f"\n【Title】: {title}")
                                print(f"【File Path】: {file_path}")
                                print("テキスト取得完了。形態素解析中...\n")

                                # 形態素解析で単語を抽出
                                extracted_words = extract_words(text)
                                print(f"抽出された単語数: {len(extracted_words)}")

                                # Word2Vecで類似度を計算
                                print(f"\n「{target_word}」との類似度が高い単語:")
                                for word in extracted_words:
                                    similarity = calculate_similarity(word, target_word, word_vectors)
                                    if similarity is not None and similarity > 0.3:  # しきい値0.3
                                        print(f"{word}: {similarity:.4f}")
                                return  # 完全一致したタイトルの処理が完了したら終了

                        except json.JSONDecodeError:
                            continue
            except Exception as e:
                print(f"エラー: {file_path} - {e}")

    print(f"\n検索完了: タイトルが「{exact_title}」と完全一致する記事は見つかりませんでした。")


In [ ]:

exact_title = "手錠"  # 完全一致で検索するタイトル
target_word = "手錠"  # 類似度を計算する対象単語

# 関数を実行
search_title_and_calculate_similarity(base_directory, exact_title, target_word)
